# Tarea 02: idf

### En esta tarea, se trabajará con un corpus de textos para obtener las palabras mas y menos relevantes a partir de un análisis por tf-idf. Deberás seguir los pasos de este notebook

In [1]:
"""
Paso 1) 
Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", 
y generar un nuevo archivo que SOLO contenga dichas preguntas.
NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y 
generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura

Paso 2) 
Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
traducción automático utilizando Google Sheets y Google Translate)
Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)

Paso 3) 
Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
OBLIGATORIAMENTE los siguientes preprocesamientos:
- Lematización de todas las palabras
- Filtrado de StopWords
- Pasar todo a minúsculas

Paso 4) 
Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
y mostrarlo en pantalla

Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
al promedio de TODOS los idfs de la tabla obtenida

Paso 6)
Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla

Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
relevantes

NOTA: Recuerda que deberás de entregar 2 archivos, un .csv con los 2,000 textos originales y sus traducciones
y un .ipynb con todo el procedimiento realizado con sus respectivos comentarios, y DocStrings
IMPORTANTE: Todo el proceso deberá realizarse por medio de métodos, NO se aceptará programación estructurada,
por lo que, por ejemplo, deberá haber un método para filtrar StopWords, otro para obtener el promedio de 
idf de todo el conjunto de palabras, etc. 
"""

'\nPaso 1) \nDel archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", \ny generar un nuevo archivo que SOLO contenga dichas preguntas.\nNOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y \ngenerar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación\ny se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura\n\nPaso 2) \nTraducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de \ntraducción automático utilizando Google Sheets y Google Translate)\nSe deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las \ntraducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)\n\nPaso 3) \nCon este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar \nOBLIGATORI

In [2]:
# Recuerda que todos los métodos que utilices deberán de contar con el formato DocString
# como en el ejemplo que se muestra a continuación:
# Ejemplo de formato Docstrings:

# def NombreFuncion(arg1, arg2, arg3):
#     """
#     Este método sirve para... utilizando... y devuelve...
    
#     Args:
#         string arg1: Esta es una cadena de texto que...
#         int arg 2: Es un número entero que se usa para...
#         dict arg 3: Diccionario que sirve para...

#     Returns:
#         string: Cadena del texto ya corregido...
#         int: El la cantidad de correcciones realizadas...
#     """

#     # Aquí debe de ir la lógica de la función (Después de la documentación)
#     Texto = ""
#     corr = 5
    
#   return Texto, corr

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")


def extract_and_save_titles(input_file, output_file, num_rows=2000):
    """
    Extract a specified number of rows from the "Title" column of a CSV file and save to a new CSV file.

    Parameters:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the extracted titles as a new CSV file.
        num_rows (int, optional): Number of rows to extract from the "Title" column. Default is 2000.
    """
    df = pd.read_csv(input_file, encoding="latin1")
    df_titles = pd.DataFrame(df.head(num_rows)["Title"])
    df_titles.to_csv(output_file, index=False)


class TextAnalysis:
    def __init__(self, filepath):
        self.filepath = filepath
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = nltk.corpus.stopwords.words("spanish")
        self.vectorizer = TfidfVectorizer(use_idf=True)
        self.read_data()
        self.preprocess_translated_text()
        self.calculate_idf()
        self.filter_below_average_idf()

    def read_data(self):
        self.df = pd.read_csv(self.filepath)["Translated_Texts"]

    def preprocess_translated_text(self):
        def process(text):
            words = nltk.word_tokenize(text)
            words = [
                self.lemmatizer.lemmatize(word.lower())
                for word in words
                if word.lower() not in self.stop_words
            ]
            return " ".join(words)

        self.df = self.df.apply(process)

    def calculate_idf(self):
        tfidf_matrix = self.vectorizer.fit_transform(self.df)
        features = self.vectorizer.get_feature_names_out()
        idf = self.vectorizer.idf_
        self.idf_dict = dict(zip(features, idf))
        self.average_idf = np.mean(idf)
        self.idf_df = pd.DataFrame(
            list(self.idf_dict.items()), columns=["Word", "IDF Value"]
        )

    def filter_below_average_idf(self):
        def filter_below_average_idf(text):
            words = text.split()
            return " ".join(
                [
                    word
                    for word in words
                    if self.idf_dict.get(word, 0) >= self.average_idf
                ]
            )

        self.below_average_idf = self.df.apply(filter_below_average_idf) # CHECAR SI NO DA UNA SOLA

    def calculate_filtered_idf(self):
        filtered_tfidf_matrix = self.vectorizer.fit_transform(self.below_average_idf)
        features = self.vectorizer.get_feature_names_out()
        idf = self.vectorizer.idf_
        self.idf_dict = dict(zip(features, idf))
        self.filtered_df = pd.DataFrame(
            list(self.idf_dict.items()), columns=["Word", "IDF Value"]
        )

    def top_bottom_words(self, top_n=10, bottom_n=20):
        sorted_idf = sorted(self.idf_dict.items(), key=lambda x: x[1], reverse=True)
        top_words = sorted_idf[:top_n]
        bottom_words = sorted_idf[-bottom_n:]
        df_top_words = pd.DataFrame(top_words, columns=["Word", "IDF Value"])
        df_bottom_words = pd.DataFrame(bottom_words, columns=["Word", "IDF Value"])
        return df_top_words, df_bottom_words


[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
if __name__ == "__main__":
    """
    Paso 1)
    Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title",
    y generar un nuevo archivo que SOLO contenga dichas preguntas.
    NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y
    generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
    y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura
    """
    extract_and_save_titles("Questions.csv", "Questions_2000.csv")

    """
    Paso 2) 
    Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
    traducción automático utilizando Google Sheets y Google Translate)
    Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
    traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)
    """
    # In TranslatedQuestions.csv file

    """
    Paso 3) 
    Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
    OBLIGATORIAMENTE los siguientes preprocesamientos:
    - Lematización de todas las palabras
    - Filtrado de StopWords
    - Pasar todo a minúsculas
    """
    analysis = TextAnalysis("TranslatedQuestions.csv")

    """
    Paso 4) 
    Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
    y mostrarlo en pantalla
    """
    print("Initial IDF Table:")
    analysis.calculate_idf()
    print(analysis.idf_df)

    """
    Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
    al promedio de TODOS los idfs de la tabla obtenidaa
    """
    analysis.filter_below_average_idf()

    """
    Paso 6)
    Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla
    """
    print("\nFiltered IDF Table:")
    analysis.calculate_filtered_idf()
    print(analysis.filtered_df)
    

    """
    Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
    relevantes
    """
    top, bottom = analysis.top_bottom_words()
    print("\nTop 10 Words:")
    print(top)
    print("\nBottom 20 Words:")
    print(bottom)

    # TODO add docstrings

    # DELETE NUMBERS
    # QUITAR DE TEXTO ORIGINAL Y VOLVER A HACER TODO

Initial IDF Table:
        Word  IDF Value
0       000z   7.908255
1         04   7.908255
2       0x1a   7.908255
3         10   7.215108
4        100   7.908255
...      ...        ...
3254   única   6.991964
3255  únicas   7.908255
3256   único   7.908255
3257  únicos   7.908255
3258    útil   7.502790

[3259 rows x 2 columns]

Filtered IDF Table:
        Word  IDF Value
0       0x1a   7.908255
1        100   7.908255
2       1000   7.908255
3        104   7.908255
4       1123   7.908255
...      ...        ...
1747  índice   7.908255
1748  último   7.908255
1749  únicas   7.908255
1750   único   7.908255
1751  únicos   7.908255

[1752 rows x 2 columns]

Top 10 Words:
   Word  IDF Value
0  0x1a   7.908255
1   100   7.908255
2  1000   7.908255
3   104   7.908255
4  1123   7.908255
5    20   7.908255
6  2006   7.908255
7  2009   7.908255
8    23   7.908255
9   301   7.908255

Bottom 20 Words:
       Word  IDF Value
0     xhtml   7.908255
1        xl   7.908255
2      xlrd   7.908255
